# 04 Introduction to Fluids Module
---

## Introduction 🌊

### What is Fluids?
[![Fluids](https://img.shields.io/badge/Fluids-8CAAE6?style=flat&logo=Fluids&logoColor=black)](https://pypi.org/project/fluids/) is a free and open-source Python library used for scientific computing and technical computing.

Fluids is open-source software for engineers and technicians working in the fields of chemical, mechanical, or civil engineering. It includes modules for piping, fittings, pumps, tanks, compressible flow, open-channel flow, atmospheric properties, solar properties, particle size distributions, two phase flow, friction factors, control valves, orifice plates and other flow meters, ejectors, relief valves, and more.

The fluids library is designed to be a low-overhead, lightweight repository of engineering knowledge and utilities that relate to fluid dynamics.

Fluids was originally tightly integrated with SciPy and NumPy; today they are optional components used for only a small amount of functionality which do not have pure-Python numerical methods implemented. Fluids targets Python 2.7 and up as well as PyPy2 and PyPy3. Additionally, fluids has been tested by the author to load in IronPython, Jython, and micropython.

While the routines in Fluids are normally quite fast and as efficiently coded as possible, depending on the application there can still be a need for further speed. PyPy provides a substantial speed boost of 6-12 times for most methods. Fluids also supports Numba, a powerful accelerator that works well with NumPy. The Numba interface to fluids also makes it easy to multithread execution as well, avoiding Python GIL issue.

Fluids runs on all operating systems which support Python, is quick to install, and is free of charge. Fluids is designed to be easy to use while still providing powerful functionality. If you need to perform some fluid dynamics calculations, give fluids a try.

In [92]:
import fluids #Importing fluids 
import math #Importing mathematics 
import thermo #Importing thermodynamics for properties

## Problem 1

Water at 30 degrees Celsius flows through a 20 m length of 50 mm plastic (smooth wall) pipe, at a flow rate of 200 L/min.

Calculate the Reynolds number and friction factor.

In [93]:
from fluids.units import  * # * means all fxn are now imported
from thermo.units import Stream # stream
from math import * # * means all fxn are now imported

In [94]:
Q = 0.2*u.m**3/u.min #Defining flow rate
Q

0.2 <Unit('meter ** 3 / minute')>

In [95]:
T = 30*u.degC #Defining temperature
T

30 <Unit('degree_Celsius')>

In [96]:
P = 2*u.bar # assumed pressure in the line
P

2 <Unit('bar')>

In [97]:
water = Stream('water', T=T, P=P, Q=Q) #Defining a variable named water and calling properties from thermo package
water

<Stream, components=['water'], mole fractions=[1.0], mass flow=3.3166445200262986 kg/s, mole flow=184.10174696292805 mol/s, T=303.15 K, P=200000 Pa>

In [98]:
NPS, Di, Do, t = nearest_pipe(Di=0.05*u.m) #Calling data from pipe database
NPS, Di, Do, t

(2 <Unit('dimensionless')>,
 0.05248 <Unit('meter')>,
 0.0603 <Unit('meter')>,
 0.00391 <Unit('meter')>)

In [99]:
v = Q/(pi/4*Di**2) #Defining velocity Volumetirc flow rate / Cross sectional area
v #Velocity is hypothetical

92.45969608160561 <Unit('meter / minute')>

In [100]:
Re = Reynolds(D=Di, rho=water.rho, mu=water.mu, V=v)
print('Reynolds number = %s' %Re)

Reynolds number = 98175.54427341053 dimensionless


In [101]:
fd = friction_factor(Re=Re, eD=_roughness['Glass']/Di)
print('Darcy friction factor = %s' %fd)

Darcy friction factor = 0.018212892385739768 dimensionless


## Problem 2 

Fuel oil at a specific gravity of 0.815 (kinematic viscosity of 2.7 centistokes) flows at 2 inch, schedule 40 steel pipe 100 foot long at a rate of 2 US gallons/second.

Calculate the pressure drop in bars and psi.

In [104]:
#Defining inputs
SG = 0.815 # Specific gravity

In [105]:
rho = SG*999.1*u.kg/u.m**3 # Converting into density
rho

814.2665 <Unit('kilogram / meter ** 3')>

In [106]:
nu = 2.7*u.centistokes # Kinematic viscosity
nu

2.7 <Unit('centistokes')>

In [107]:
mu = nu_mu_converter(rho,  nu=nu) # Converting into abs viscosity
mu

0.0021985195500000007 <Unit('pascal * second')>

In [108]:
Q = 2*u.gal/u.s # Flow rate
Q

2.0 <Unit('gallon / second')>

In [109]:
L = 100*u.foot # Length
L

100 <Unit('foot')>

In [110]:
NPS, D_pipe, Do_pipe, t = nearest_pipe(Di=2*u.inch) #Calling from piping database
NPS, D_pipe, Do_pipe, t

(2 <Unit('dimensionless')>,
 0.05248 <Unit('meter')>,
 0.0603 <Unit('meter')>,
 0.00391 <Unit('meter')>)

In [112]:
v = Q/(pi/4*D_pipe**2) #Defining velocity Q/A
v

924.5969608160561 <Unit('gallon / meter ** 2 / second')>

In [150]:
Re = Reynolds(rho=rho, mu=mu, D=D_pipe, V=v) #Calculating reynolds numbers
Re

40670.93582448257 <Unit('dimensionless')>

In [151]:
fd = friction_factor(Re=Re, eD=0.0018*u.inch/D_pipe) #Calculating friction drop
fd

0.022452729812545924 <Unit('dimensionless')>

In [152]:
K_friction = K_from_f(fd=fd, L=L, D=D_pipe) #Calculating friction
print('Darcy friction factor = %s' %fd)

Darcy friction factor = 0.022452729812545924 dimensionless


In [153]:
dP = dP_from_K(K=K_friction, rho=rho, V=v) #Calculating pressure drop
print('Pressure drop = %s' %dP.to(u.Pa))
print('In imperial, pressure drop = %s' %dP.to(u.psi))

Pressure drop = 363.5041410814 pascal
In imperial, pressure drop = 0.05272181827800907 pound_force_per_square_inch


## Problem 3

Lube oil at 40 C is flowing at the rate of 600 barrels/hr through 60 meters of 8 inch schedule 40 pipe. Find the total line pressure drop in bar.

Specific gravity : 0.90 at 40 C

Viscosity : 450 cP at 40 C

In [140]:
SG = 0.9 #Defining specific gravity
SG

0.9

In [141]:
rho = SG*999.1*u.kg/u.m**3 #Defining rho
rho

899.19 <Unit('kilogram / meter ** 3')>

In [142]:
nu = 450*u.centipoise #Defining visocity
nu

450 <Unit('centipoise')>

In [143]:
Q = 600*u.barrel/u.hr #Defining flow rate
Q

600.0 <Unit('barrel / hour')>

In [144]:
L = 60*u.meter #Defining length
L

60 <Unit('meter')>

In [145]:
NPS, D_pipe, Do_pipe, t = nearest_pipe(Di=8*u.inch) #Calling piping data base
NPS, D_pipe, Do_pipe, t

(10 <Unit('dimensionless')>,
 0.25446 <Unit('meter')>,
 0.273 <Unit('meter')>,
 0.009269999999999999 <Unit('meter')>)

In [146]:
v = Q/(pi/4*D_pipe**2) #Calculating velocity 
v

11798.378467727003 <Unit('barrel / hour / meter ** 2')>

In [147]:
Re = Reynolds(rho=rho, mu=mu, D=D_pipe, V=v) #Calculating reynolds number
Re

40670.93582448257 <Unit('dimensionless')>

In [148]:
fd = friction_factor(Re=Re, eD=0.0018*u.inch/D_pipe) #Calculating friction drop
print('Darcy friction factor = %s' %fd)

Darcy friction factor = 0.022452729812545924 dimensionless


In [149]:
K_friction = K_from_f(fd=fd, L=L, D=D_pipe) #Calculating k friction
dP = dP_from_K(K=K_friction, rho=rho, V=v) #Calculating pressure drop
print('Pressure drop = %s' %dP.to(u.Pa))
print('In metric, pressure drop = %s' %dP.to(u.bar))

Pressure drop = 363.5041410814 pascal
In metric, pressure drop = 0.0036350414108140003 bar
